# create.ipynb
This notebook leverages GalactICS to make initial conditions files ready for simulation in GIZMO.

In [1]:
import os
import subprocess

import initial_conditions as ic

In [44]:
# CONFIG
SCRATCH = '/scratch/gpfs/chainje'
HOME = '/home/chainje'
SIM = f'{SCRATCH}/sidm_dm_only/sgr'
GADGETCONVERTERS = f'{HOME}/GalactICSPackage_Mar2019/GadgetConverters/Programs'

SIDM = True

In [ ]:
# Set up directory
os.makedirs(f'{SIM}/galactics', exist_ok=True)
os.makedirs(f'{SIM}/converter', exist_ok=True)
os.makedirs(f'{SIM}/output', exist_ok=True)

First, we need to make the GalactICS input files. This is handled by code in `initial_conditions.py`. One can specify the parameters necessary to make a parameter file manually, but the local `mw.py` and `sgr.py` files define all the relevant parameters needed. To use one of these, simply uncomment its import statement.

In [ ]:
# Make GalactICS input files
# from mw import *
from sgr import *

h, d = ic.make_params(M_NFW, c, M_halo, N_halo, R_200, r_H, M_disk, N_disk, b_0, c_0, M_bulge, N_bulge, a,
                      sigma_h_fudge=sigma_h_fudge)
w = ic.Writer(halo=h, disk=d, folder=f'{SIM}/galactics')
w.write()

With the input files made, we make use of the `run_galactics.sh` script to actually run GalactICS. If GalactICS runs successfully, we should be able to decode the captured stdout and determine the mass of the halo. If GalactICS does not run successfully, we are not likely to get the halo mass back out, so we will print the stderr.

In [25]:
# Run GalactICS
result = subprocess.run(['sh', 'run_galactics.sh', f'{SIM}'], capture_output=True)
output = result.stdout.decode('utf-8').split('\n')
for line in output:
    if 'Halo mass' in line:
        print(f"Halo mass: {float(line.split()[3]) * 2.325e9:.3e} M_sol")
        break
else:
    print(result.stderr.decode('utf-8').split('\n'))

Halo mass: 1.303e+10 M_sol


Now that we've run GalactICS, we need to convert the initial conditions files into a format that GIZMO understands. This is done with GalactICS's GadgetConverters module. First, we make a file called `converter.in` that specifies the number of galaxy components (disk, halo, disk2, etc.), the path to the GalactICS output, and the output paths. This is finicky; if modifying (for instance, to add a bulge or a second disk), be careful to leave extra newlines in all the same spaces. The final result is a GIZMO binary initial conditions file at `{SIM}/data.ic`.

In [45]:
# Run GadgetConverters

## make a local converter.in
f = open('converter.in', 'w')
f.write('1\t2\n\n') # number of galaxies, max number of components
f.write(f'{SIM}/galactics/\n\n') # path to galactics output
f.write('2\nhalo,\t1\ndisk,\t2\n')
f.write(f'{SIM}/converter/ignore.out\n\n')
f.write(f'{SIM}/converter/ascii.out\n\n')
f.write(f'{SIM}/converter/partIDs.out\n')
f.write('0.\n0.\n0\n0\n0\n0.\n0.\n0.\n0.\n')
f.close()

## clean the sim converter folder
subprocess.run(['rm',f'{SIM}/converter/*'])

## run galcombine to get ascii.out
results = subprocess.run([f'{GADGETCONVERTERS}/GalCombine', 'converter.in'], capture_output=True)
# print(results.stdout.decode('utf-8'))
# print(results.stderr.decode('utf-8'))

## convert ascii.out to a GIZMO binary
p = subprocess.Popen([f'{GADGETCONVERTERS}/ascii2gadget_gasIni'],
                     stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
outs, errs = p.communicate(bytes(os.linesep.join([f'{SIM}/converter/ascii.out',f'{SIM}/data.ic']), 'utf-8'))
# print(outs.decode('utf-8'))
# print(errs.decode('utf-8'))

subprocess.run(['rm','converter.in'])

 Combining Galaxies
 Number of galaxies to combine           1
 The path to the GalactICS data files is /scratch/gpfs/chainje/sidm_dm_only/sgr/galactics/
 number of Components in           1 Galaxy           2
 halo,	1                                                                                             
 Component File and Particle Type: /scratch/gpfs/chainje/sidm_dm_only/sgr/galactics/halo           1
 disk,	2                                                                                             
 Component File and Particle Type: /scratch/gpfs/chainje/sidm_dm_only/sgr/galactics/disk           2
 Output centered Galaxy           1 to /scratch/gpfs/chainje/sidm_dm_only/sgr/converter/ignore.out
 Output ascii combined galaxy to /scratch/gpfs/chainje/sidm_dm_only/sgr/converter/ascii.out
 Output Particle ID file for later separation to /scratch/gpfs/chainje/sidm_dm_only/sgr/converter/partIDs.out
 Reading in the various GalactICS Components
           1           1           1  

Lastly, we prep the directory for GIZMO simulation. We do this by making a parameterfile in the simulation directory with the correct paths hardcoded in the directory. This is done by modifying a local copy of the parameterfile and simply copying it to the simulation directory. Then, a sample SLURM batch script is created in the simulation directory. GIZMO can be run by simply running `sbatch run_gizmo.sh` from the simulation directory!

Note also that, since we now have our `data.ic` initial conditions file, the `galactics/` and `converter/` directories (and all their contents) can be safely removed. However, it could be useful to hang on to at least the GalactICS input files if you ever need to double check the parameters used.

In [52]:
# Get everything ready for GIZMO

## update the local gizmo.param file with correct filepaths
with open("gizmo.param", "r") as f:
    lines = f.readlines()
for i in range(len(lines)):
    if 'InitCondFile' in lines[i]:
        lines[i] = f'InitCondFile\t    {SIM}/data.ic\n'
    elif 'OutputDir' in lines[i]:
        lines[i] = f'OutputDir\t    {SIM}/output\n'
    elif 'DM_InteractionCrossSection' in lines[i]:
        lines[i] = f'DM_InteractionCrossSection    {"2" if SIDM else "0"}\n'
with open("gizmo.param", "w") as f:
    f.writelines(lines)

## copy local paramfile to the sim directory
subprocess.run(['cp','gizmo.param',f'{SIM}'])

## populate a sample batch script
with open(f"{SIM}/run_gizmo.sh", "w") as f:
    f.write("#!/bin/bash\n")
    f.write("#SBATCH --job-name=sim\n")
    f.write("#SBATCH -p all\n")
    f.write("#SBATCH --nodes=1\n")
    f.write("#SBATCH --exclusive\n")
    f.write("#SBATCH --n-tasks-per-node=25\n")
    f.write("#SBATCH --cpus-per-task=1\n")
    f.write("#SBATCH --mem=50G\n")
    f.write("#SBATCH --time=0-00:20:00\n")
    f.write("#SBATCH --mail-type=all\n")
    f.write("#SBATCH --mail-user=chainje@princeton.edu\n\n")
    f.write("module purge\n")
    f.write("module load openmpi/gcc/1.10.2/64 gsl/2.4 fftw/gcc/openmpi-1.10.2/3.3.4 hdf5/gcc/1.8.16\n")
    f.write(f"srun {HOME}/gizmo-{'sidm' if SIDM else 'public'}/GIZMO {SIM}/gizmo.param\n")